In [9]:
import cv2
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
#from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image

In [10]:
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing import image

# Define the base directory
base_dir = "AIOralCancerProject"

# Create an empty list to store the images and labels
X = []
y = []

# Get the class labels from the directory names
class_labels = os.listdir(base_dir)

# Iterate through each directory and store the images and labels
for i, label in enumerate(class_labels):
    img_dir = os.path.join(base_dir, label)
    for file in os.listdir(img_dir):
        # Check if file is an image
        if file.endswith(".jpg") or file.endswith(".png"):
            # Load the image
            
            img = Image.open(os.path.join(img_dir, file))
            img = img.resize((300, 300))
            img = np.array(img)
            #img = img / 255.0

            # Append the image to the list of images
            X.append(img)
            # Append the corresponding label to the list of labels
            y.append(i)

In [11]:
# Shuffle the data
#X, y = shuffle(X, y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.reshape(-1,300,300,3)
X_test = X_test.reshape(-1,300,300,3)

In [13]:
from keras.preprocessing.image import ImageDataGenerator

In [14]:
# Define the data generator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [15]:
# Fit the generator to the training data
datagen.fit(X_train)

In [16]:
from keras.utils import to_categorical

# One-hot encode the labels
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [17]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Freeze the base model layers
base_model.trainable = False

# Create a new model on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Dropout(0.5)(x)
predictions = Dense(16, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
#history = model.fit(X_train, y_train_encoded, batch_size=64, epochs=30, validation_data=(X_test, y_test_encoded))


#history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=64,  epochs=30, validation_data=(X_test, y_test)))

history = model.fit_generator(datagen.flow(X_train, y_train_encoded, batch_size=64), epochs=30, validation_data=(X_test, y_test_encoded))

2023-01-27 12:15:42.546410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-27 12:15:42.546481: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mistakenly-eager-jawfish): /proc/driver/nvidia/version does not exist
2023-01-27 12:15:42.547479: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


43941136/43941136 [==============================] - 2s 0us/step


/tmp/ipykernel_104116/3509303453.py:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train, y_train_encoded, batch_size=64), epochs=30, validation_data=(X_test, y_test_encoded))


Epoch 1/30
35/35 [==============================] - 192s 5s/step - loss: 2.4765 - accuracy: 0.2235 - val_loss: 2.2830 - val_accuracy: 0.2932
Epoch 2/30
35/35 [==============================] - 171s 5s/step - loss: 2.1474 - accuracy: 0.3201 - val_loss: 2.1476 - val_accuracy: 0.3165
Epoch 3/30
35/35 [==============================] - 161s 5s/step - loss: 2.0342 - accuracy: 0.3570 - val_loss: 2.0605 - val_accuracy: 0.3723
Epoch 4/30
35/35 [==============================] - 157s 4s/step - loss: 1.9427 - accuracy: 0.3723 - val_loss: 2.0118 - val_accuracy: 0.3723
Epoch 5/30
35/35 [==============================] - 153s 4s/step - loss: 1.8651 - accuracy: 0.3907 - val_loss: 1.9438 - val_accuracy: 0.4011
Epoch 6/30
35/35 [==============================] - 152s 4s/step - loss: 1.8024 - accuracy: 0.4182 - val_loss: 1.9008 - val_accuracy: 0.4083
Epoch 7/30
35/35 [==============================] - 155s 4s/step - loss: 1.7594 - accuracy: 0.4429 - val_loss: 1.8935 - val_accuracy: 0.4245
Epoch 8/30
35

In [19]:
import tensorflow as tf
tf.saved_model.save(model, 'AIOralCancerProject')

INFO:tensorflow:Assets written to: AIOralCancerProject/assets


INFO:tensorflow:Assets written to: AIOralCancerProject/assets


In [20]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print('Test accuracy:', test_acc)

18/18 [==============================] - 24s 1s/step - loss: 1.6836 - accuracy: 0.4982
Test accuracy: 0.4982014298439026
